In [17]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


# 라이브러리 import

In [18]:
!pip install pymysql
!pip install -q sklearn
!pip install tensorflow_addons 
!pip install pyyaml h5py  # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다


In [19]:
!mkdir -p saved_model

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import feature_column
from tensorflow.keras import layers
import tensorflow_addons as tfa


import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [21]:
import pymysql


# 데이터 전처리

In [22]:
conn = pymysql.connect(
    user = 'orlab',
    passwd = 'orlabghkdlxld123!',
    host = 'orlab.cl9jlcc8q0pr.ap-northeast-2.rds.amazonaws.com',
    db = 'Dataset_final',
    charset = 'utf8'
)
curs = conn.cursor(pymysql.cursors.DictCursor)
curs1 = conn.cursor(pymysql.cursors.DictCursor)

curs.execute("select * from Dataset_4_2016")

df = curs.fetchall()
df = pd.DataFrame(df)
df

,index,Project_ID,Year,N_of_SCI,N_of_Paper,N_Patent_App,N_Patent_Reg,N_of_Korean_Patent,N_of_Inter_Patent,N_of_Patent,Multi_Year,RnD_Org,Region,STP_Code_11,STP_Code_1_Weight,STP_Code_21,STP_Code_2_Weight,Application_Area_1,Application_Area_1_Weight,Application_Area_2,Application_Area_2_Weight,Application_Area_3,Application_Area_3_Weight,Green_Tech,SixT_2,Econ_Social,National_Strategy_2,RnD_Stage,Cowork_Cor,Cowork_Uni,Cowork_Inst,Cowork_Abroad,Cowork_etc,Log_RnD_Fund,Log_Duration,Comm_Success,Comm_Success_1,Comm_Success_2,Comm_Success_Code1_4,Comm_Success_Code2_5,Comm_Success_Code3_6
0,0,1055000474,2013.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,1,OC03,100.0,None,0.0,X99,100.0,None,0.0,None,0.0,999,70000,13,60000,4,N,N,N,N,N,21.365120,5.897157,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1055000475,2013.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,1,OC03,100.0,None,0.0,X99,100.0,None,0.0,None,0.0,999,70000,13,60000,4,N,N,N,N,N,19.568083,5.897157,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1055000476,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,1,OC03,100.0,None,0.0,X99,100.0,None,0.0,None,0.0,999,70000,13,60000,4,N,N,N,N,N,21.365120,5.897157,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1055000477,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,1,SB12,100.0,None,0.0,X99,100.0,None,0.0,None,0.0,999,70000,13,60000,4,N,N,N,N,N,19.673444,5.808145,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1055000478,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2,19,ZZ,0.0,None,0.0,X99,100.0,None,0.0,None,0.0,999,70000,13,60000,4,N,N,N,N,N,17.504390,4.804029,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192874,192874,9991006121,2016.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,1,2,8,LC03,50.0,LC03,30.0,X01,50.0,X99,30.0,X02,20.0,999,20200,11,40100,2,N,N,N,N,N,20.723266,6.302621,0.0,0.0,0.0,0.0,0.0,0.0
192875,192875,9991006122,2016.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,8,1,LC03,100.0,None,0.0,X02,100.0,None,0.0,None,0.0,999,20200,4,60000,3,N,N,N,N,Y,21.920324,6.302621,0.0,0.0,0.0,0.0,0.0,0.0
192876,192876,9991006124,2016.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5,8,EB02,80.0,EA03,20.0,Y08,60.0,Y09,40.0,None,0.0,999,30200,7,10500,3,Y,N,Y,N,N,20.482467,6.998511,0.0,0.0,0.0,0.0,0.0,0.0
192877,192877,9991006125,2016.0,0.0,0.0,2.0,2.0,4.0,0.0,4.0,2,5,8,ED10,100.0,None,0.0,Y05,100.0,None,0.0,None,0.0,999,30200,7,10500,3,Y,N,Y,N,N,20.430236,6.998511,1.0,1.0,0.0,0.0,1.0,0.0


In [23]:
df.drop(['index','Project_ID'],axis = 1, inplace = True)

# feature columns 생성

In [24]:

feature_columns = []

for header in ['Log_RnD_Fund','Log_Duration','N_of_SCI','N_of_Paper','N_Patent_App','N_Patent_Reg','N_of_Korean_Patent','N_of_Inter_Patent',
               'N_of_Patent','STP_Code_1_Weight','STP_Code_2_Weight','Application_Area_1_Weight','Application_Area_2_Weight','Application_Area_3_Weight']:
    feature_columns.append(feature_column.numeric_column(header, dtype=tf.dtypes.float64, ))

# 범주형 열(Categorical column)은 특정 문자열을 수치형으로 매핑하여 전달

#Year
df["Year"] = df["Year"].apply(str)
Year = tf.feature_column.categorical_column_with_vocabulary_list(
      'Year', df['Year'].unique() )
Year_hot = tf.feature_column.indicator_column(Year)
feature_columns.append(Year_hot)


#Multi_Year
df["Multi_Year"] = df["Multi_Year"].apply(str)
Multi_Year = tf.feature_column.categorical_column_with_vocabulary_list(
      'Multi_Year', df['Multi_Year'].unique() )
Multi_Year_one_hot = tf.feature_column.indicator_column(Multi_Year)
feature_columns.append(Multi_Year_one_hot)


#RnD_Org
df["RnD_Org"] = df["RnD_Org"].apply(str)
RnD_Org = tf.feature_column.categorical_column_with_vocabulary_list(
      'RnD_Org', df['RnD_Org'].unique() )
RnD_Org_one_hot = tf.feature_column.indicator_column(RnD_Org)
feature_columns.append(RnD_Org_one_hot)

#Region
df["Region"] = df["Region"].apply(str)

Region = tf.feature_column.categorical_column_with_vocabulary_list(
      'Region', df['Region'].unique() )
Region_one_hot = tf.feature_column.indicator_column(Region)
feature_columns.append(Region_one_hot)

#STP_Code_11
df["STP_Code_11"] = df["STP_Code_11"].apply(str)

STP_Code_11 = tf.feature_column.categorical_column_with_vocabulary_list(
      'STP_Code_11', df['STP_Code_11'].unique() )
STP_Code_11_one_hot = tf.feature_column.indicator_column(STP_Code_11)
feature_columns.append(STP_Code_11_one_hot)


#STP_Code_21
df["STP_Code_21"] = df["STP_Code_21"].apply(str)

STP_Code_21 = tf.feature_column.categorical_column_with_vocabulary_list(
      'STP_Code_21', df['STP_Code_21'].unique() )
STP_Code_21_one_hot = tf.feature_column.indicator_column(STP_Code_21)
feature_columns.append(STP_Code_21_one_hot)



#Application_Area_1
df["Application_Area_1"] = df["Application_Area_1"].apply(str)

Application_Area_1 = tf.feature_column.categorical_column_with_vocabulary_list(
      'Application_Area_1', df['Application_Area_1'].unique() )
Application_Area_1_one_hot = tf.feature_column.indicator_column(Application_Area_1)
feature_columns.append(Application_Area_1_one_hot)



#Application_Area_2
df["Application_Area_2"] = df["Application_Area_2"].apply(str)

Application_Area_2 = tf.feature_column.categorical_column_with_vocabulary_list(
      'Application_Area_2', df['Application_Area_2'].unique() )
Application_Area_2_one_hot = tf.feature_column.indicator_column(Application_Area_2)
feature_columns.append(Application_Area_2_one_hot)

#Application_Area_3
df["Application_Area_3"] = df["Application_Area_3"].apply(str)

Application_Area_3 = tf.feature_column.categorical_column_with_vocabulary_list(
      'Application_Area_3', df['Application_Area_3'].unique() )
Application_Area_3_one_hot = tf.feature_column.indicator_column(Application_Area_3)
feature_columns.append(Application_Area_3_one_hot)



#Green_Tech
df["Green_Tech"] = df["Green_Tech"].apply(str)

Green_Tech = tf.feature_column.categorical_column_with_vocabulary_list(
      'Green_Tech', df['Green_Tech'].unique() )
Green_Tech_one_hot = tf.feature_column.indicator_column(Green_Tech)
feature_columns.append(Green_Tech_one_hot)



#SixT_2
df["SixT_2"] = df["SixT_2"].apply(str)
SixT_2 = tf.feature_column.categorical_column_with_vocabulary_list(
      'SixT_2', df['SixT_2'].unique() )
SixT_2_one_hot = tf.feature_column.indicator_column(SixT_2)
feature_columns.append(SixT_2_one_hot)


#Econ_Social
df["Econ_Social"] = df["Econ_Social"].apply(str)
Econ_Social = tf.feature_column.categorical_column_with_vocabulary_list(
      'Econ_Social', df['Econ_Social'].unique() )
Econ_Social_one_hot = tf.feature_column.indicator_column(Econ_Social)
feature_columns.append(Econ_Social_one_hot)

#National_Strategy_2
df["National_Strategy_2"] = df["National_Strategy_2"].apply(str)
National_Strategy_2 = tf.feature_column.categorical_column_with_vocabulary_list(
      'National_Strategy_2', df['National_Strategy_2'].unique() )
National_Strategy_2_one_hot = tf.feature_column.indicator_column(National_Strategy_2)
feature_columns.append(National_Strategy_2_one_hot)

#RnD_Stage
df["RnD_Stage"] = df["RnD_Stage"].apply(str)
RnD_Stage = tf.feature_column.categorical_column_with_vocabulary_list(
      'RnD_Stage', df['RnD_Stage'].unique() )
RnD_Stage_one_hot = tf.feature_column.indicator_column(RnD_Stage)
feature_columns.append(RnD_Stage_one_hot)

#Cowork_Cor
df["Cowork_Cor"] = df["Cowork_Cor"].apply(str)
Cowork_Cor = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_Cor', df['Cowork_Cor'].unique() )
Cowork_Cor_one_hot = tf.feature_column.indicator_column(Cowork_Cor)
feature_columns.append(Cowork_Cor_one_hot)

#Cowork_Uni
df["Cowork_Uni"] = df["Cowork_Uni"].apply(str)
Cowork_Uni = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_Uni', df['Cowork_Uni'].unique() )
Cowork_Uni_one_hot = tf.feature_column.indicator_column(Cowork_Uni)
feature_columns.append(Cowork_Uni_one_hot)

#Cowork_Inst
df["Cowork_Inst"] = df["Cowork_Inst"].apply(str)
Cowork_Inst = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_Inst', df['Cowork_Inst'].unique() )
Cowork_Inst_one_hot = tf.feature_column.indicator_column(Cowork_Inst)
feature_columns.append(Cowork_Inst_one_hot)

#Cowork_Abroad
df["Cowork_Abroad"] = df["Cowork_Abroad"].apply(str)
Cowork_Abroad = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_Abroad', df['Cowork_Abroad'].unique() )
Cowork_Abroad_one_hot = tf.feature_column.indicator_column(Cowork_Abroad)
feature_columns.append(Cowork_Abroad_one_hot)

#Cowork_etc
df["Cowork_etc"] = df["Cowork_etc"].apply(str)
Cowork_etc = tf.feature_column.categorical_column_with_vocabulary_list(
      'Cowork_etc', df['Cowork_etc'].unique() )
Cowork_etc_one_hot = tf.feature_column.indicator_column(Cowork_etc)
feature_columns.append(Cowork_etc_one_hot)




In [25]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [26]:
feature_columns

[NumericColumn(key='Log_RnD_Fund', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='Log_Duration', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_SCI', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_Paper', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_Patent_App', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_Patent_Reg', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_Korean_Patent', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_Inter_Patent', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='N_of_Patent', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='STP_Code_1_Weight', sh

In [27]:
len(df['Green_Tech'].value_counts())

60

# train, test, val split

In [28]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

In [29]:
# train
x_train = train_df.loc[:,['Year', 'N_of_SCI', 'N_of_Paper', 'N_Patent_App', 'N_Patent_Reg',
       'N_of_Korean_Patent', 'N_of_Inter_Patent', 'N_of_Patent', 'Multi_Year',
       'RnD_Org', 'Region', 'STP_Code_11', 'STP_Code_1_Weight', 'STP_Code_21',
       'STP_Code_2_Weight', 'Application_Area_1', 'Application_Area_1_Weight',
       'Application_Area_2', 'Application_Area_2_Weight', 'Application_Area_3',
       'Application_Area_3_Weight', 'Green_Tech', 'SixT_2', 'Econ_Social',
       'National_Strategy_2', 'RnD_Stage', 'Cowork_Cor', 'Cowork_Uni',
       'Cowork_Inst', 'Cowork_Abroad', 'Cowork_etc', 'Log_RnD_Fund',
       'Log_Duration']]
y_train = train_df.loc[:,['Comm_Success']]
x_train = dict(x_train)

# val
x_val = val_df.loc[:,['Year', 'N_of_SCI', 'N_of_Paper', 'N_Patent_App', 'N_Patent_Reg',
       'N_of_Korean_Patent', 'N_of_Inter_Patent', 'N_of_Patent', 'Multi_Year',
       'RnD_Org', 'Region', 'STP_Code_11', 'STP_Code_1_Weight', 'STP_Code_21',
       'STP_Code_2_Weight', 'Application_Area_1', 'Application_Area_1_Weight',
       'Application_Area_2', 'Application_Area_2_Weight', 'Application_Area_3',
       'Application_Area_3_Weight', 'Green_Tech', 'SixT_2', 'Econ_Social',
       'National_Strategy_2', 'RnD_Stage', 'Cowork_Cor', 'Cowork_Uni',
       'Cowork_Inst', 'Cowork_Abroad', 'Cowork_etc', 'Log_RnD_Fund',
       'Log_Duration']]
y_val = val_df.loc[:,['Comm_Success']]
x_val = dict(x_val)

# test
x_test = test_df.loc[:,['Year', 'N_of_SCI', 'N_of_Paper', 'N_Patent_App', 'N_Patent_Reg',
       'N_of_Korean_Patent', 'N_of_Inter_Patent', 'N_of_Patent', 'Multi_Year',
       'RnD_Org', 'Region', 'STP_Code_11', 'STP_Code_1_Weight', 'STP_Code_21',
       'STP_Code_2_Weight', 'Application_Area_1', 'Application_Area_1_Weight',
       'Application_Area_2', 'Application_Area_2_Weight', 'Application_Area_3',
       'Application_Area_3_Weight', 'Green_Tech', 'SixT_2', 'Econ_Social',
       'National_Strategy_2', 'RnD_Stage', 'Cowork_Cor', 'Cowork_Uni',
       'Cowork_Inst', 'Cowork_Abroad', 'Cowork_etc', 'Log_RnD_Fund',
       'Log_Duration']]
y_test = test_df.loc[:,['Comm_Success']]
x_test = dict(x_test)


#모델생성

## 825

In [30]:
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import ModelCheckpoint

In [31]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      tfa.metrics.F1Score(num_classes=1, threshold=0.5)

]


In [32]:
cb = tf.keras.callbacks.EarlyStopping( monitor='val_loss', min_delta = 0,  patience=500, verbose=1, mode='min', baseline=None, restore_best_weights=True )

def creat_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns=feature_columns),
    tf.keras.layers.Dense(units=1000, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(units=300, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(units=300, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss= tfa.losses.SigmoidFocalCrossEntropy(gamma = 2, alpha=0.825),
                metrics=METRICS)
  
  return model


model = creat_model()


model.fit(
    x_train, y_train,
    batch_size=256,
    epochs=300,
    validation_data=(x_val, y_val),
    callbacks=cb
)


print('-----------------------테스트 결과---------------------------')
loss, BinaryAccuracy ,Precision, Recall,AUC,F1Score= model.evaluate(x_test,y_test)

print('')
print('')
# 모델 저장 경로
model.save('/gdrive/My Drive/Colab Notebooks/DNN/alpha/modelsave/프로젝트_성공_0.825')

Epoch 1/300
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
482/483 [============================>.] - ETA: 0s - loss: 0.0428 - accuracy: 0.7579 - precision: 0.3113 - recall: 0.7701 - auc: 0.8303 - f1_score: 0.4434WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Year': <tf.Tensor 'ExpandDims_32:0' shape=(None, 1) dtype=string>, 'N_of_SCI': <tf.Tensor 'ExpandDims_22:0' shape=(None, 1) dtype=float64>, 'N_of_Paper': <tf.Tensor 'ExpandDims_20:0' shape=(None, 1) dtype=float64>, 'N_Patent_App': <tf.Tensor 'ExpandDims_16:0' shape=(None, 1) dtype=float64>, 'N_Patent_Reg': <tf.Tensor 'ExpandDims_17:0' shape=(None, 1) dtype=float64>, 'N_of_Korean_Patent': <tf.Tensor 'ExpandDims_19:0' shape=(None, 1) dtype=float64>, 'N_of_Inter_Patent': <tf.Tensor 'ExpandDims_18:0' shape=(None, 1) dtype=float64>, 'N_of_Patent': <tf.Tensor 'ExpandDims_21:0' shape=(None, 1) d

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


INFO:tensorflow:Assets written to: /gdrive/My Drive/Colab Notebooks/DNN/alpha/modelsave/프로젝트_성공_0.825/assets


INFO:tensorflow:Assets written to: /gdrive/My Drive/Colab Notebooks/DNN/alpha/modelsave/프로젝트_성공_0.825/assets
